In [1]:
from xbos import get_client
from xbos.services.pundat import DataClient, timestamp, make_dataframe
from xbos.services.hod import HodClientHTTP
%matplotlib inline
import pandas as pd

In [2]:
client = get_client() # BOSSWAVE client (low level) 
dataclient = DataClient(client)
hodclient = HodClientHTTP("http://ciee.cal-sdb.org")

Saw [ucberkeley] archiver 6 seconds 992.762 ms ago


In [3]:
# timesteps
start = 'now'
end = 'now -21d'

In [4]:
# get building meter
# get bosswave URIs 
query = """SELECT ?dev ?uuid WHERE {
  ?dev rdf:type/rdfs:subClassOf* brick:Electric_Meter .
  ?dev bf:uuid ?uuid .
};
"""
meter_uuids = [x['?uuid'] for x in hodclient.do_query(query)]
print meter_uuids[0]
meter_timeseries = make_dataframe(dataclient.data_uuids(meter_uuids, start, end))
meter_df = meter_timeseries.values()[0]

4d6e251a-48e1-3bc0-907d-7d5440c34bb9


In [5]:
# get bosswave URIs and uuids for thermostat statuses
query = """SELECT ?dev ?p ?uuid WHERE {
  ?dev rdf:type/rdfs:subClassOf* brick:Thermostat .
  ?dev bf:hasPoint ?p .
  ?p bf:uuid ?uuid .
  ?p rdf:type brick:Thermostat_Status .
};
"""
state_uuids = [x['?uuid'] for x in hodclient.do_query(query)]
print state_uuids
# these UUIDs are the timeseries for what the thermostat is doing
state_timeseries = make_dataframe(dataclient.data_uuids(state_uuids, start, end))

[u'dfb2b403-fd08-3e9b-bf3f-18c699ce40d6', u'7e543d07-16d1-32bb-94af-95a01f4675f9', u'5e55e5b1-007b-39fa-98b6-ae01baa6dccd', u'187ed9b8-ee9b-3042-875e-088a08da37ae']


In [6]:
def estimate_power_usage(d):
    diffs = d['value'].diff()
    turnon = d[diffs > 0]
    turnoff = d[diffs < 0]
    window = pd.Timedelta('20s')
    guesses = []
    for ts in turnon.index:
        # find biggest energy difference in a [-window, +window] slice around the state transition
        guess = meter_df.loc[ts-window:ts+window].diff().max().value
        if pd.isnull(guess) or guess == 0: continue
        guesses.append(guess)
    for ts in turnoff.index:
        # find biggest energy difference in a [-window, +window] slice around the state transition
        # negate it because this is the off->on signal, and use min()
        guess = -meter_df.loc[ts-window:ts+window].diff().min().value
        if pd.isnull(guess) or guess == 0: continue
        guesses.append(guess)
    guess = pd.np.median(guesses)
    if pd.isnull(guess): return 0.0
    return guess

# get info on thermostats
def get_hvac_zone(uuid):
    query = """SELECT ?zone ?uuid WHERE {{
      ?zone rdf:type brick:HVAC_Zone .
     ?tstat bf:hasPoint/bf:uuid "{0}" .
     ?tstat bf:controls/bf:feeds ?zone .
}};""".format(uuid)
    res = hodclient.do_query(query)[0]
    return res['?zone']
    

In [7]:
for uuid, df in state_timeseries.items():
    usage = estimate_power_usage(df)
    zone = get_hvac_zone(uuid)
    print 'Thermostat ({0}) (Zone {1}) has estimated avg power consumption {2} W'.format(uuid, zone, usage)

Thermostat (5e55e5b1-007b-39fa-98b6-ae01baa6dccd) (Zone NorthZone) has estimated avg power consumption 5000.0 W
Thermostat (7e543d07-16d1-32bb-94af-95a01f4675f9) (Zone EastZone) has estimated avg power consumption 80.0 W
Thermostat (dfb2b403-fd08-3e9b-bf3f-18c699ce40d6) (Zone SouthZone) has estimated avg power consumption 5160.0 W
Thermostat (187ed9b8-ee9b-3042-875e-088a08da37ae) (Zone CentralZone) has estimated avg power consumption 4960.0 W
